# Парсим мемы в питоне: как обойти серверную блокировку. 


<img align="center" src="pictures/captain.jpg
" height="300" width="300"> 

# 1. Вламываемся в мемохранилище

## 1.1. Что мы хотим получить

Итак, мы хотим распарсить [knowyourmeme.com](http://knowyourmeme.com) и получить кучу разных переменных: 

- **Name** – название мема,
- **Origin_year** – год его создания,
- **Views** – число просмотров,
- **About** – текстовое описание мема,
- ** и многие другие** 

Более того, мы хотим сделать это без вот этого всего: 

<img align="center" src="pictures/2.png" height="600" width="600">

После скачивания и чистки данных от мусора, можно будет заняться строительством моделей. Например, попытаться предсказать популярность мема по его параметрам. Но это все позже, а сейчас познакомимся с парой определений:

* **Парсер** — это скрипт, который грабит информацию с сайта
* **Краулер** — это часть парсера, которая бродит по ссылкам
* **Краулинг** — это переход по страницам и ссылкам
* **Скрапинг** — это сбор данных со страниц
* **Парсинг** — это сразу и краулинг и скрапинг! 


## 1.2.  Что такое HTML 

**HTML (HyperText Markup Language)**  — это такой же язык разметки как Markdown или LaTeX. Он является стандартным для написания различных сайтов. Команды в таком языке называются **тегами**. Если открыть абсолютно любой сайт, нажать на правую кнопку мышки, а после нажать `View page source`, то перед вами предстанет HTML скелет этого сайта. 

Можно увидеть, что HTML-страница это ни что иное как набор вложенных тегов. Можно заметить, например, следующие теги:

- `<title>` – заголовок страницы
- `<h1>…<h6>` – заголовки разных уровней
- `<p>` – абзац (paragraph)
- `<div>` – выделения фрагмента документа с целью изменения вида содержимого
- `<table>` – прорисовка таблицы 
- `<tr>` – разделитель для строк в таблице 
- `<td>` – разделитель для столбцов в таблице
- `<b>` – устанавливает жирное начертание шрифта

Обычно команда `<...>` открывает тег, а  `</...>` закрывает его. Все, что находится между этими двумя командами, подчиняется правилу, которое диктует тег. Например, все, что находится между `<p>` и  `</p>` — это отдельный абзац.   

Теги образуют своеобразное дерево с корнем в теге `<html>` и разбивают страницу на разные логические кусочки. У каждого тега могут быть свои потомки (дети) — те теги, которые вложены в него, и свои родители. 

Например, HTML-древо страницы может выглядеть вот так:

    <html>
    <head> Заголовок </head>
    <body>
        <div> 
            Первый кусок текста со своими свойствами
        </div>
        <div>
            Второй кусок текста
                <b>
                    Третий, жирный кусок
                </b>
        </div>
        Четвёртый кусок текста        
    </body>
    </html>            
    
    
<img align="center" src="pictures/tree.png" height="500" width="500"> 

Можно работать с этим html как с текстом, а можно как с деревом. Обход этого дерева и есть парсинг веб-страницы. Мы всего лишь будем находить нужные нам узлы среди всего этого разнообразия и забирать из них информацию!

Вручную обходить эти деревья не очень приятно, поэтому есть специальные языки для обхода деревьев.

- [CSS-селектор](https://ru.wikibooks.org/wiki/CSS/%D0%A1%D0%B5%D0%BB%D0%B5%D0%BA%D1%82%D0%BE%D1%80%D1%8B) (это когда мы ищем элемент страницы по паре ключ, значение)
- [XPath](https://ru.wikipedia.org/wiki/XPath) (это когда мы прописываем путь по дереву вот так: /html/body/div[1]/div[3]/div/div[2]/div)
- Всякие разные библиотеки для всяких разных языков, например, BeautifulSoup для питона. Именно эту библиотеку мы и будем использовать. 

## 1.3. Наш первый запрос

Доступ к веб-станицам позволяет получать модуль `requests`. Подгрузим его. За компанию подгрузим ещё парочку дельных пакетов. 

In [1]:
import requests      # Библиотека для отправки запросов
import numpy as np   # Библиотека для матриц, векторов и линала
import pandas as pd  # Библиотека для табличек 
import time          # Библиотека для времени

Для наших благородных исследовательских целей нужно собрать данные по каждому мему с соответствующей ему страницы. Но для начала нужно получить адреса этих страниц. Поэтому открываем основную страницу со всеми выложенными мемами. Выглядит она следующим образом:

<img align="center" src="pictures/memes_main.png" height="500" width="500"> 

Отсюда мы и будем тащить ссылки на каждый из перечисленных мемов. Сохраним в переменную `page_link` адрес основной страницы и откроем её при помощи библиотеки `requests`.

In [2]:
page_link = 'http://knowyourmeme.com/memes/all/page/1'

In [3]:
response = requests.get(page_link)
response

<Response [403]>

А вот и первая проблема! Обращаемся к [главному источнику знаний](https://en.wikipedia.org/wiki/HTTP_403) и выясняем, что 403-я ошибка выдается сервером, если он доступен и способен обрабатывать запросы, но по некоторым личным причинам отказывается это делать. 

<img align="center" src="pictures/403.jpg" height="350" width="350"> 

Попробуем выяснить, почему. Для этого проверим, как выглядел финальный запрос, отправленный нами на сервер, а конкретнее - как выглядел наш User-Agent в глазах сервера.

In [4]:
for key, value in response.request.headers.items():
    print(key+": "+value)

User-Agent: python-requests/2.22.0
Accept-Encoding: gzip, deflate
Accept: */*
Connection: keep-alive


Похоже, мы недвусмысленно дали понять серверу, что мы сидим на питоне и используем библиотеку requests под версией 2.14.2. Скорее всего, это вызвало у сервера некоторые подозрения относительно наших благих намерений и он решил нас безжалостно отвергнуть. Для сравнения, можно посмотреть, как выглядят request-headers у здорового человека:

<img align="center" src="pictures/good_headers.png" height="800" width="800"> 

Очевидно, что нашему скромному запросу не тягаться с таким обилием мета-информации, которое передается при запросе из обычного браузера. К счастью, никто нам не мешает притвориться человечными и пустить пыль в глаза сервера при помощи генерации фейкового юзер-агента. Библиотек, которые справляются с такой задачей, существует очень и очень много, лично мне больше всего нравится [`fake-useragent`](https://pypi.python.org/pypi/fake-useragent). При вызове метода из различных кусочков будет генерироваться рандомное сочетание операционной системы, спецификаций и версии браузера, которые можно передавать в запрос:

In [5]:
# подгрузим один из методов этой библиотеки
from fake_useragent import UserAgent

In [6]:
UserAgent().chrome

'Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/29.0.1547.2 Safari/537.36'

In [7]:
response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
response

<Response [200]>

<img align="center" src="pictures/200.jpeg" height="350" width="350">

Замечательно, наша небольшая маскировка сработала и обманутый сервер покорно выдал благословенный 200 ответ — соединение установлено и данные получены, всё чудесно! Посмотрим, что же все-таки мы получили.

In [8]:
html = response.content

In [9]:
html[:1000]

b'<!DOCTYPE html>\n<html xmlns:fb=\'https://www.facebook.com/2008/fbml\' xmlns=\'https://www.w3.org/1999/xhtml\'>\n<head>\n<meta content=\'text/html; charset=utf-8\' http-equiv=\'Content-Type\'>\n<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"c1a6d52f38","applicationID":"31165848","transactionName":"dFdfRUpeWglTQB8GDUNKWFRLHlcJWg==","queueTime":0,"applicationTime":66,"agent":""}</script>\n<script type="text/javascript">window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var o=n[t]={exports:{}};e[t][0].call(o.exports,function(n){var o=e[t][1][n];return r(o||n)},o,o.exports)}return n[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<t.length;o++)r(t[o]);return r}({1:[function(e,n,t){function r(){}function o(e,n,t){return function(){return i(e,[c.now()].concat(u(arguments)),n?null:this,t),n?void 0:this}}var i=e("handle"),a=e(3),u=e(4),f=e("e

Выглядит неудобоваримо, как насчет сварить из этого дела что-то покрасивее? Например, красивый суп.

## 1.4. Красивый суп

<img align="center" src="pictures/soup.jpg" height="200" width="200"> 

Пакет **[bs4 , a.k.a BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/)** (тут есть гиперссылка на лучшего друга человека — документацию) был назван в честь стишка про красивый суп из Алисы в стране чудес.

Красивый суп — это совершенно волшебная библиотека, которая из сырого и необработанного HTML кода страницы выдаст вам структурированный массив данных, по которому очень удобно искать необходимые теги, классы, атрибуты, тексты и прочие элементы веб страниц.

> Пакет под названием `BeautifulSoup` — скорее всего, не то, что нам нужно. Это третья версия (*Beautiful Soup 3*), а мы будем использовать четвертую. Нужно будет установить пакет `beautifulsoup4`. Чтобы было совсем весело, при импорте нужно указывать другое название пакета — `bs4`, а импортировать функцию под названием `BeautifulSoup`. В общем, сначала легко запутаться, но эти трудности нужно преодолеть.

С необработанным XML кодом страницы пакет также работает (XML — это исковерканый и превращённый в диалект, с помощью своих команд, HTML). Для того, чтобы пакет корректно работал с XML разметкой, придётся в довесок ко всему нашему арсеналу установить пакет `xml`. 

In [10]:
from bs4 import BeautifulSoup

Передадим функции `BeautifulSoup` текст веб-страницы, которую мы недавно получили.

In [11]:
soup = BeautifulSoup(html,'html.parser') # В опции также можно указать lxml, 
                                         # если предварительно установить одноименный пакет

Получим что-то вот такое:
    
```
<!DOCTYPE html>

<html xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"c1a6d52f38","applicationID":"31165848","transactionName":"dFdfRUpeWglTQB8GDUNKWFRLHkUNWUU=","queueTime":0,"applicationTime":24,"agent":""}</script>
<script type="text/javascript">window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var o=n[t]={exports:{}};e[t][0].call(o.exports,function(n){var o=e[t][1][n];return r(o||n)},o,o.exports)}return n[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<t.length;o++)r(t[o]);return r}({1:[function(e,n,t){function r(){}function o(e,n,t){return function(){return i(e,[c.now()].concat(u(arguments)),n?null:this,t),n?void 0:this}}var i=e("handle"),a=e(2),u=e(3),f=e("ee").get("tracer"),c=e("loader"),s=NREUM;"undefined"==typeof window.newrelic&&(newrelic=s);var p=
```

Стало намного лучше, не правда ли? Что же лежит в переменной `soup`? Невнимательный пользователь, скорее всего, скажет,что ничего вообще не изменилось. Тем не менее, это не так. Теперь мы можем свободно бродить по HTML-дереву страницы, искать детей, родителей и вытаскивать их! 

Например, можно бродить по вершинам, указывая путь из тегов.

In [12]:
soup.html.head.title

<title>All Entries | Know Your Meme</title>

Можно вытащить из того места, куда мы забрели, текст с помощью метода `text`.

In [13]:
soup.html.head.title.text

'All Entries | Know Your Meme'

Более того, зная адрес элемента, мы сразу можем найти его. Например, можно сделать это по классу. Следующая команда должна найти элемент, который лежит внутри тега `a` и имеет класс `photo`.

In [14]:
obj = soup.find('a', attrs = {'class':'photo'})
obj

<a class="photo left" href="/memes/when-was-calcium-invented" target="_self"><img alt="Redditors Imagine Life Without Various Discoveries" data-src="https://i.kym-cdn.com/featured_items/icons/wide/000/010/048/Untitled-1.png" height="112" src="https://s.kym-cdn.com/assets/blank-b3f96f160b75b1b49b426754ba188fe8.gif" title="Redditors Imagine Life Without Various Discoveries" width="198"/> <div class="info abs"> <div class="c"> Redditors Imagine Life Without Various Discoveries </div> </div> </a>

Однако, вопреки нашим ожиданиям, вытащенный объект имеет класс `"photo left"`. Оказывается, `BeautifulSoup4` расценивает аттрибуты `class` как набор отдельных значений, поэтому `"photo left"` для библиотеки равносильно `["photo", "left"]`, а указанное нами значение этого класса `"photo"` входит в этот список. Чтобы избежать такой неприятной ситуации и проходов по ненужным нам ссылкам, придется воспользоваться собственной функцией и задать строгое соответствие:

In [15]:
obj = soup.find(lambda tag: tag.name == 'a' and tag.get('class') == ['photo'])
obj

<a class="photo" href="/memes/boss-makes-a-dollar-i-make-a-dime"><img alt="Boss Makes a Dollar, I Make a Dime" data-src="https://i.kym-cdn.com/entries/icons/medium/000/031/634/guy-fired-over-meme-job-work-post-facebook-cody-hidalgo-fb.png" src="https://s.kym-cdn.com/assets/blank-b3f96f160b75b1b49b426754ba188fe8.gif" title="Boss Makes a Dollar, I Make a Dime"/> <div class="entry-labels"> <span class="label label-submission"> Submission </span> </div> </a>

Полученный после поиска объект также обладает структурой bs4. Поэтому можно продолжить искать нужные нам объекты уже в нём! Вытащим ссылку на этот мем. Сделать это можно по атрибуту `href`, в котором лежит наша ссылка. 

In [16]:
obj.attrs['href']

'/memes/boss-makes-a-dollar-i-make-a-dime'

Обратите внимание, что после всех этих безумных преобразований у данных поменялся тип. Теперь они `str`. Это означет, что с ними можно работать как с текстом и пускать в ход для отсеивания лишней информации регулярные выражения. 

In [17]:
print("Тип данных до вытаскивания ссылки:", type(obj))
print("Тип данных после вытаскивания ссылки:", type(obj.attrs['href']))

Тип данных до вытаскивания ссылки: <class 'bs4.element.Tag'>
Тип данных после вытаскивания ссылки: <class 'str'>


Если несколько элементов на странице обладают указанным адресом, то метод `find` вернёт только самый первый.  Чтобы найти все элементы с таким адресом, нужно использовать метод `findAll`, и на выход будет выдан список. Таким образом, мы можем получить одним поиском сразу все объекты, содержащие ссылки на страницы с мемами.

In [18]:
meme_links = soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['photo'])
meme_links[:3]

[<a class="photo" href="/memes/boss-makes-a-dollar-i-make-a-dime"><img alt="Boss Makes a Dollar, I Make a Dime" data-src="https://i.kym-cdn.com/entries/icons/medium/000/031/634/guy-fired-over-meme-job-work-post-facebook-cody-hidalgo-fb.png" src="https://s.kym-cdn.com/assets/blank-b3f96f160b75b1b49b426754ba188fe8.gif" title="Boss Makes a Dollar, I Make a Dime"/> <div class="entry-labels"> <span class="label label-submission"> Submission </span> </div> </a>,
 <a class="photo" href="/memes/events/death-of-abu-bakr-al-baghdadi"><img alt="Death of Abu Bakr al-Baghdadi" data-src="https://i.kym-cdn.com/entries/icons/medium/000/031/633/abu.jpg" src="https://s.kym-cdn.com/assets/blank-b3f96f160b75b1b49b426754ba188fe8.gif" title="Death of Abu Bakr al-Baghdadi"/> <div class="entry-labels"> <span class="label label-submission"> Submission </span> <span class="label" style="background: #0b6a40; color: white;">Event</span> </div> </a>,
 <a class="photo" href="/memes/subcultures/overwatch-2"><img alt

Осталось очистить полученный список от мусора:

In [19]:
meme_links = [link.attrs['href'] for link in meme_links]

In [20]:
meme_links[:10]

['/memes/boss-makes-a-dollar-i-make-a-dime',
 '/memes/events/death-of-abu-bakr-al-baghdadi',
 '/memes/subcultures/overwatch-2',
 '/memes/get-that-thing-out-of-my-face',
 '/memes/events/teamtrees',
 '/memes/listen-to-your-heart',
 '/memes/abenzioshs-shiny-ponyta-video',
 '/memes/are-you-going-to-sleep',
 '/memes/epstein-didnt-kill-himself',
 '/memes/running-away-balloon']

In [21]:
len(meme_links)

16

Готово, получили ровно 16 ссылок по числу мемов на одной странице поиска. 

Хорошо, то, что можно искать элемент по его адресу, конечно же, круто, но откуда взять этот адрес? Можно установить для своего браузера какую-нибудь утилиту, позволяющую вытаскивать со страницы нужные теги, например, [selectorgadget.](http://selectorgadget.com/)

Тем не менее, этот путь не подходит для истинного самурая. Для последователей бусидо есть другой способ — искать теги для каждого нужного нам элемента вручную. Для этого придётся жать правой кнопкой мышки по окну браузера и тыкать кнопку **Inspect**. После всех этих манипуляций браузер будет выглядеть как-то вот так: 

<img align="center" src="pictures/memes_inspection.png" height="800" width="800"> 

Выскочивший кусок html, в котором находится адрес выбранного вами объекта, можно смело копировать в код и наслаждаться своей брутальностью. 

Остался последний момент. Когда мы скачаем все мемы с текущей страницы, нам нужно будет каким-то образом забраться на соседнюю. На сайте это можно делать просто пролистывая страницу с мемами вниз, javascript-функции подтянут новые мемы на текущее окно, но сейчас трогать эти функции не хочется.

Обычно, все параметры, которые мы устанавливаем на сайте для поиска, отображаются на структуре хрефа. Мемы не являются исключением. Если мы хотим получить первую порцию мемов, мы должны будем обратиться к сайту по ссылке 

                `http://knowyourmeme.com/memes/all/page/1`


Если мы захотим получить вторую поцию с шестнадцатью мемами, нам придётся немного видоизменить ссылку, а именно заменить номер страницы на 2.


                `http://knowyourmeme.com/memes/all/page/2`
 
Таким незатейливым образом мы сможем пройтись по всем страницам и ограбить мемохранилище. 

Наконец, обернем в красивую функцию все-все манипуляции, проделанные выше:

In [22]:
def getPageLinks(page_number):
    """
        Возвращает список ссылок на мемы, полученный с текущей страницы
        
        page_number: int/string
            номер страницы для парсинга
            
    """
    # составляем ссылку на страницу поиска
    page_link = 'http://knowyourmeme.com/memes/all/page/{}'.format(page_number)
    
    # запрашиваем данные по ней
    response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
    
    if not response.ok:
        # если сервер нам отказал, вернем пустой лист для текущей страницы
        return [] 
    
    # получаем содержимое страницы и переводим в суп
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    
    # наконец, ищем ссылки на мемы и очищаем их от ненужных тэгов
    meme_links = soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['photo'])
    meme_links = ['http://knowyourmeme.com' + link.attrs['href'] for link in meme_links]
    
    return meme_links

Протестируем функцию и убедимся, что всё хорошо

In [23]:
meme_links = getPageLinks(1)
meme_links[:2]

['http://knowyourmeme.com/memes/cinderblock',
 'http://knowyourmeme.com/memes/boss-makes-a-dollar-i-make-a-dime']

In [24]:
meme_links = getPageLinks(2)
meme_links[:2]

['http://knowyourmeme.com/memes/i-am-ferdinand-von-aegir',
 'http://knowyourmeme.com/memes/people/cole-carrigan']

Отлично, функция работает и теперь мы теоретически можем достать ссылки на все $17171$ мем, для чего нам придется пройтись по $\frac{17171}{16} \approx 1074$ страницам. Прежде чем расстраивать сервер таким количеством запросов, посмотрим, как доставать всю необходимую информацию о конкретном меме. 

## 1.5 Финальная подготовка к грабежу

По аналогии со ссылками можно вытащить что угодно. Для этого надо сделать несколько шагов: 

1. Открываем страничку с мемом
2. Находим любым способом тег для нужной нам информации
3. Пихаем всё это в красивый суп
4. ......
5. Profit 

Для закрепления информации в голове любознательного читателя, вытащим число просмотров мема.

А в качестве примера возьмем самый популярный на этом сайте мем - Doge, набравший более 12 миллионов просмотров по состоянию на 1 января 2018 года. 

Сама страница, с которой мы будем доставать дорогую нашему исследовательскому сердцу информацию выглядит следуюшим образом:

<img align="center" src="pictures/doge_main.png" height="600" width="600"> 

Как и прежде, для начала сохраним ссылку на страницу в переменную и вытащим по ней контент.

In [25]:
meme_page = 'http://knowyourmeme.com/memes/doge'

response = requests.get(meme_page, headers={'User-Agent': UserAgent().chrome})

html = response.content
soup = BeautifulSoup(html,'html.parser')

Посмотрим, как можно вытащить статистику просмотров, комментариев, а также числа загруженных видео и фото, связанных с нашим мемов. Всё это добро хранится справа вверху под тэгами `"dd"` и с классами  `"views"`, `"videos"`, `"photos"` и `"comments"`

In [26]:
views = soup.find('dd', attrs={'class':'views'})
views

<dd class="views" title="13,072,407 Views">
<a href="/memes/doge" rel="nofollow">13,072,407</a>
</dd>

In [27]:
views = views.find('a').text
views

'13,072,407'

In [28]:
views = int(views.replace(',', ''))
views

13072407

Снова запихнём всё это в небольшую функцию.

In [50]:
def getStats(soup, stats):
    """
        Возвращает очищенное число просмотров/коментариев/...
        
        soup: объект bs4.BeautifulSoup 
            суп текущей страницы
            
        stats: string
            views/videos/photos/comments
            
    """
    try:
        obj = soup.find('dd', attrs={'class':stats})
        obj = obj.find('a').text
        obj = int(obj.replace(',', ''))
    except:
        obj=None
    
    return obj

Всё готово! 

In [51]:
views = getStats(soup, stats='views')
videos = getStats(soup, stats='videos')
photos = getStats(soup, stats='photos')
comments = getStats(soup, stats='comments')

print("Просмотры: {}\nВидео: {}\nФото: {}\nКомментарии: {}".format(views, videos, photos, comments))

Просмотры: 13072407
Видео: 62
Фото: 1663
Комментарии: 917


Еще из интересного и исследовательского —  достанем дату и время добавления мема. Если посмотреть на страницу в браузере, можно подумать, что максимум информации, который мы можем вытащить - это число лет, прошедших с момента публикации —  `Added 4 years ago by NovaXP`. Однако мы так просто сдаваться не будем, полезем в кишки html и откопаем там кусок, ответственный за эту надпись:

<img align="center" src="pictures/no_shit.jpg" height="300" width="300"> 

<img align="center" src="pictures/html_time_ago.png" height="600" width="600"> 

Ага! Вот и подробности по дате добавления, с точностью до минуты. Элементарно

In [31]:
date = soup.find('abbr', attrs={'class':'timeago'}).attrs['title']
date

'2018-10-23T01:43:59-04:00'

На самом деле, парсеры — дело непредсказуемое. Часто страницы, которые мы парсим, имеют очень неоднородну структуру. Например, если мы парсим мемы, на части страниц может быть указано описание, а на части нет. Как только код впервые натыкается на отсутствие описания, он выдаёт ошибку и останавливается. Чтобы нормально собрать все данные, приходится прописывать исключения. Вроде бы, хранилище мемов хорошо оборудовано и никаких внештатных ситуаций происходить не должно. Тем не менее, очень не хочется проснуться утром и увидеть, что код сделал 20 итераций, нарвался на ошибку и отрубился.  Чтобы такого не произошло, можно, например, использовать конструкцию `try - except` и просто обрабатывать неугодные нам ошибки. Про исключения можно почитать [на просторах интернета](https://pythonworld.ru/tipy-dannyx-v-python/isklyucheniya-v-python-konstrukciya-try-except-dlya-obrabotki-isklyuchenij.html). В нашем же случае до ошибки можно и не доводить, а предварительно проверять, есть ли необходимый элемент на странице или нет при помощи обычного `if - else`, и уже после этого пытаться его распарсить.

Например, мы хотим вытащить статус мема, для этого найдем окружающие его тэги:

In [32]:
properties = soup.find('aside', attrs={'class':'left'})
meme_status = properties.find("dd")
meme_status

<dd>
Confirmed
</dd>

Дальше нужно вытащить из тэгов текст, а после обрубить все лишние пробелы. 

In [33]:
meme_status.text.strip()

'Confirmed'

Однако, если неожиданно выяснится, что у мема нет статуса, метод `find` вернёт пустоту. Метод `text`, в свою очередь, не сможет найти в тэгах текст и выдаст ошибку. Чтобы обезопасить себя от таких пустот, можно прописать исключение или `if - else`. Так как в текущем меме статус все-таки есть, нарочно зададим его как пустой объект, чтобы проверить, что ошибка поймается в обоих случаях

In [34]:
# Делай раз! Ищем статус мема, но не находим его
meme_status = None

# Делай два! Пытаемся вытащить его...

# ... с исключениями
try:
    print(meme_status.text.strip()) 
# Ежели возникает ошибка, статус не найден, выдаём пустоту.
except:
    print("Exception")
    
    
# ... с проверкой на пустой элемент
if meme_status:
    print(meme_status.text.strip())
else:
    print("Empty")

Exception
Empty


Такой код позволяет обезопасить себя от ошибок в коде. В данном случае, мы можем переписать всю конструкцию с `if - else` в виде одной удобной строки. Эта строка проверит полон ли респонса `meme_status` и ежели нет, то выдаст пустоту.

In [35]:
# снова найдем настоящий статус
properties = soup.find('aside', attrs={'class':'left'})
meme_status = properties.find("dd")

meme_status = "" if not meme_status else meme_status.text.strip()
print(meme_status)

Confirmed


По аналогии можно вытащить всю остальную информацию со страницы.

In [36]:
def getProperties(soup):
    """
        Возвращает список (tuple) с названием, статусом, типом, 
        годом и местом происхождения и тэгами
        
        soup: объект bs4.BeautifulSoup 
            суп текущей страницы
    
    """
    # название - идёт с самым большим заголовком h1, легко найти
    meme_name = soup.find('section', attrs={'class':'info'}).find('h1').text.strip()
    
    # достаём все данные справа от картинки 
    properties = soup.find('aside', attrs={'class':'left'})
    
    # статус идет первым - можно не уточнять класс
    meme_status = properties.find("dd")
    # oneliner, заменяющий try-except: если тэга нет в properties, вернётся объект NoneType,
    # у которого аттрибут text отсутствует, и в этом случае он заменится на пустую строку
    meme_status = "" if not meme_status else meme_status.text.strip()
    
    # тип мема - обладает уникальным классом
    meme_type = properties.find('a', attrs={'class':'entry-type-link'})
    meme_type = "" if not meme_type else meme_type.text 
    
    # год происхождения первоисточника можно найти после заголовка Year, 
    # находим заголовок, определяем родителя и ищем следущего ребенка - наш раздел
    meme_origin_year = properties.find(text='\nYear\n')
    meme_origin_year = "" if not meme_origin_year else meme_origin_year.parent.find_next()
    meme_origin_year = meme_origin_year.text.strip()
    
    # сам первоисточник
    meme_origin_place = properties.find('dd', attrs={'class':'entry_origin_link'})
    meme_origin_place = "" if not meme_origin_place else meme_origin_place.text.strip()
    
    # тэги, связанные с мемом
    meme_tags = properties.find('dl', attrs={'id':'entry_tags'}).find('dd')
    meme_tags = "" if not meme_tags else meme_tags.text.strip()
    
    return meme_name, meme_status, meme_type, meme_origin_year, meme_origin_place, meme_tags

In [37]:
getProperties(soup)

('Doge',
 'Confirmed',
 'Animal',
 '2013',
 'Tumblr',
 'animal, dog, shiba inu, shibe, such doge, super shibe, japanese, super, tumblr, much, very, many, comic sans, photoshop meme, such, shiba, shibe doge, doges, dogges, reddit, comic sans ms, tumblr meme, hacked, bitcoin, dogecoin, shitposting, stare, canine')

Свойства мема собрали. Теперь собираем по аналогии его текстовое описание. 

In [38]:
def getText(soup):
    """
        Возвращает текстовые описания мема
        
        soup: объект bs4.BeautifulSoup 
            суп текущей страницы
            
    """
    
    # достаём все тексты под картинкой
    body = soup.find('section', attrs={'class':'bodycopy'})
    
    # раздел about (если он есть), должен идти первым, берем его без уточнения класса
    meme_about = body.find('p')
    meme_about = "" if not meme_about else meme_about.text
    
    # раздел origin можно найти после заголовка Origin или History, 
    # находим заголовок, определяем родителя и ищем следущего ребенка - наш раздел
    meme_origin = body.find(text='Origin') or body.find(text='History')
    meme_origin = "" if not meme_origin else meme_origin.parent.find_next().text
    
    # весь остальной текст (если он есть) можно запихнуть в одно текстовое поле
    if body.text:
        other_text = body.text.strip().split('\n')[4:]
        other_text = " ".join(other_text).strip()
    else:
        other_text = ""
        
    return meme_about, meme_origin, other_text

In [39]:
meme_about, meme_origin, other_text = getText(soup)

print("О чем мем:\n{}\n\nПроисхождение:\n{}\n\nОстальной текст:\n{}...\n"\
      .format(meme_about, meme_origin, other_text[:200]))

О чем мем:
Doge is a slang term for "dog" that is primarily associated with pictures of Shiba Inus (nicknamed "Shibe") and internal monologue captions on Tumblr. These photos may be photoshopped to change the dog's face or captioned with interior monologues in Comic Sans font.

Происхождение:
The use of the misspelled word "doge" to refer to a dog dates back to June 24th, 2005, when it was mentioned in an episode of Homestar Runner's puppet show. In the episode titled "Biz Cas Fri 1"[2], Homestar calls Strong Bad his "d-o-g-e" while trying to distract him from his work.

Остальной текст:
Identity On February 23rd, 2010, Japanese kindergarten teacher Atsuko Sato posted several photos of her rescue-adopted Shiba Inu dog Kabosu to her personal blog.[38] Among the photos included a peculi...



Наконец, создадим функцию, возвращающую всю информацию по текущему мему

In [40]:
def getMemeData(meme_page):
    """
        Запрашивает данные по странице, возвращает обработанный словарь с данными
        
        meme_page: string
            ссылка на страницу с мемом
    
    """
    
    # запрашиваем данные по ссылке
    response = requests.get(meme_page, headers={'User-Agent': UserAgent().chrome})
    
    if not response.ok:
        # если сервер нам отказал, вернем статус ошибки 
        return response.status_code
    
    # получаем содержимое страницы и переводим в суп
    html = response.content
    soup = BeautifulSoup(html,'html.parser')

    # используя ранее написанные функции парсим информацию
    views = getStats(soup=soup, stats='views')
    videos = getStats(soup=soup, stats='videos')
    photos = getStats(soup=soup, stats='photos')
    comments = getStats(soup=soup, stats='comments')

    # дата
    date = soup.find('abbr', attrs={'class':'timeago'}).attrs['title']

    # имя, статус, и т.д.
    meme_name, meme_status, meme_type, meme_origin_year, meme_origin_place, meme_tags =\
    getProperties(soup=soup)

    # текстовые поля
    meme_about, meme_origin, other_text = getText(soup=soup)

    # составляем словарь, в котором будут хранится все полученные и обработанные данные
    data_row = {"name":meme_name, "status":meme_status, 
                "type":meme_type, "origin_year":meme_origin_year, 
                "origin_place":meme_origin_place,
                "date_added":date, "views":views, 
                "videos":videos, "photos":photos, "comments":comments, "tags":meme_tags,
                "about":meme_about, "origin":meme_origin, "other_text":other_text}

    return data_row

In [41]:
data_row = getMemeData('http://knowyourmeme.com/memes/doge')

А теперь подготовим табличку, чтобы в неё записывать всё ~~награбленные~~ честно полученные данные, добавим в неё первую полученную строку и полюбуемся на результат

In [42]:
final_df = pd.DataFrame(columns=['name', 'status', 'type', 'origin_year', 'origin_place',
                                 'date_added', 'views', 'videos', 'photos', 'comments', 
                                 'tags', 'about', 'origin', 'other_text'])

In [43]:
final_df = final_df.append(data_row, ignore_index=True)

In [44]:
final_df

,name,status,type,origin_year,origin_place,date_added,views,videos,photos,comments,tags,about,origin,other_text
0,Doge,Confirmed,Animal,2013,Tumblr,2018-10-23T01:43:59-04:00,13072407,62,1663,917,"animal, dog, shiba inu, shibe, such doge, supe...","Doge is a slang term for ""dog"" that is primari...","The use of the misspelled word ""doge"" to refer...","Identity On February 23rd, 2010, Japanese kind..."


Первый мем оказался в наших рукак. Еще раз убедимся что всё работает — пройдемся по списку из ссылок на мемы, полученных ранее в перменной `meme_links`.

In [52]:
from tqdm import tqdm_notebook

In [61]:
for meme_link in tqdm_notebook(meme_links):
    data_row = getMemeData(meme_link)
    final_df = final_df.append(data_row, ignore_index=True)

In [62]:
final_df.head()

,name,status,type,origin_year,origin_place,date_added,views,videos,photos,comments,tags,about,origin,other_text
0,Doge,Confirmed,Animal,2013,Tumblr,2018-10-23T01:43:59-04:00,13072407,62,1663,917,"animal, dog, shiba inu, shibe, such doge, supe...","Doge is a slang term for ""dog"" that is primari...","The use of the misspelled word ""doge"" to refer...","Identity On February 23rd, 2010, Japanese kind..."
1,I Am Ferdinand Von Aegir,Submission,Copypasta,2019,Fire Emblem: Three Houses,2019-10-25T12:55:28-04:00,136,0,0,0,"fire emblem, three houses, ferdinand von aegir",I Am Ferdinand Von Aegir is a quote from the c...,"In Fire Emblem: Three Houses, released July 26...",Spread Various Examples Search Interest tren...
2,Cole Carrigan,Submission,Influencer,2013,YouTube,2019-10-27T05:23:04-04:00,3651,0,0,9,"makeup, drama, ace family, team ten, youtuber,...",Cole Carrigan is an American Beauty vlogger an...,,"On July 21st, 2019, Cole Carrigan uploaded one..."
3,Quagmire Toilet,Submission,Pop Culture Reference,2014,Family Guy,2019-10-28T10:13:18-04:00,23897,0,6,32,"quagmire, shitposting, family guy, finger elev...",Quagmire Toilet refers to an image from an epi...,"The image comes from the Family Guy episode ""B...",Spread The image saw some light parodies in th...
4,Former french President Sarkozy reverse smile,Submission,Pop Culture Reference,2019,http://nordpresse.be/operation-sarkozy-a-la-fr...,2019-10-25T09:04:55-04:00,67,0,0,0,"sarkozy, surprised, digusted, funny face, read...","On the 10/18/2019, Vincent Flibustier, the cre...",,


In [63]:
final_df.shape

(22, 14)

Отлично! Всё работает, мемы качаются, данные наполняются и всё было бы хорошо, если бы не одно но — количество запросов, которое нам придётся сделать, чтобы всё получить.

# 2.  Прячемся от стражников

## 2.1 Когда работающий код больше не работает

Вот он! Тот самый момент абсолютного триумфа, когда код дописан и всё, что нам, мирным собирателям, остаётся — запустить наш код на одну ночку. Кажется, что через страсть мы преобрели силу. Запускаем наш код по всем $1075$ страницам с мемами. На всякий случай обернём наш цикл в `try-except`. Мало ли что там с этими мемами бывает. 

In [54]:
# Немного красивых циклов. При желании пакет можно отключить и 
# удалить команду tqdm_notebook из всех циклов
from tqdm import tqdm_notebook


final_df = pd.DataFrame(columns=['name', 'status', 'type', 'origin_year', 'origin_place',
                                 'date_added', 'views', 'videos', 'photos', 'comments', 
                                 'tags', 'about', 'origin', 'other_text'])

for page_number in tqdm_notebook(range(1075), desc='Pages'):
    # собрали хрефы с текущей страницы
    meme_links = getPageLinks(page_number)  
    for meme_link in tqdm_notebook(meme_links, desc='Memes', leave=False):
        # иногда с первого раза страничка не парсится
        for i in range(5):
            try:
                # пытаемся собрать по мему немного даты
                data_row = getMemeData(meme_link)           
                # и закидываем её в таблицу
                final_df = final_df.append(data_row, ignore_index=True)  
                # если всё получилось - выходим из внутреннего цикла
                break
            except:
                # Иначе, пробуем еще несколько раз, пока не закончатся попытки
                print('AHTUNG! parsing once again:', meme_link)
                continue

Сон был прекрасным! Солнце только-только взошло из-за горизонта, мы уже бежим за компьютер смотреть мемы и видим, что огромное число мемов не скачалось.

<img align="center" src="pictures/work_or_not.jpg" height="500" width="500">

Конечно же, вполне естественной реакцией будет нажать на первую же ссылку, перейти в мемохранилище и увидеть, что нас забанили.

<img align="center" src="pictures/memes_ban.png" height="600" width="600"> 

Все наши реквесты остались без респонсов. 

## 2.2 Тор - сын Одина

Иногда серверу надоедает общаться с одним и тем же человеком, делающим кучу запросов и сервер банит его. К сожалению, не только у людей запас терпения ограничен.

Приходится маскироваться. Для такой маскировки можно использовать разные способы, более того, один из них мы уже использовали, когда притворились человеком в нашем `request-header`. Для текущей же задачи, когда нас вероломно заблокировали по IP, нужно искать способы помощнее, чтобы иметь возможность этот IP менять. Конечно, как вариант можно было бы использовать прокси-сервера, тогда мы бы имели в запасе некоторое количество разных IP адресов, которые можно подставлять по мере "забанивания". Однако в этом подходе есть пара проблем: первая - нужно где-то раздобыть эти прокси, вторая - а что если ограниченного числа адресов нам не хватит и нужно больше?

В таком случае лучше всего нам подойдёт [Tor](https://www.torproject.org/). Вопреки пропагандируемому мнению, Tor используется не только преступниками, педофилами и прочими нехорошими террористами. Это, мягко говоря, далеко не так, и мы, мирные собиратели данных, являемся тому подтверждением. Всем прелестям, связанным с работой Tor, можно было бы посвятить несколько больших статеек, что собственно говоря уже и сделано. Подробнее про это можно почитатать по следующим ссылкам: 

* [Как работает Tor](https://geektimes.ru/post/277578/)
* [Методы анонимности в сети](https://habrahabr.ru/post/204266/)
* [Прокси-сервер с помощью Tor](https://habrahabr.ru/company/etagi/blog/315002/)

Мы же ограничимся только функциональной частью, а именно без углубления в детали опишем шаги, которые нужно предпринять для того, чтобы использовать возможности Tor для обхода блокировки. Для начала полюбуемся на свой ip-адрес. Для этого сделаем get-запрос к сайту, который возвращяет наш IP

In [64]:
def checkIP():
    ip = requests.get('http://checkip.dyndns.org').content
    soup = BeautifulSoup(ip, 'html.parser')
    print(soup.find('body').text)

In [65]:
checkIP()

Current IP Address: 82.130.48.29


Заменить свой ip через Tor можно двумя путями. Простой - через браузер, а сложный - через небольшие махинации с настройками.

Скачаем браузерный [tor,](https://www.torproject.org/download/download) чтобы лёгкий путь был совсем прост. Для сложного пути в довесок к браузеру поставим tor через консоль. 
- Linux - нам поможет команда `apt-get install tor`, 
- Mac - сделаем это [в рамках brew](https://www.torproject.org/docs/tor-doc-osx.html.en), `brew install tor`. 
- Windows - нам поможет установка другой операционной системы. 

## 2.3 Путь первый 

Теперь запускаем свежескачанный браузер и оставляем его открытым.  Менять ip нам поможет библиотека `PySocks`. Конечно же, её нужно установить, скопировав в терминал `pip3 install PySocks`.  

Браузер тора по умолчанию использует порт номер 9150. В питоне при помощи библиотек socks и socket можно задать дефолтный порт для подключения. В результате текущая сессия будет использовать именно этот порт при отправке любого запроса, а значит – запросы будут посылаться из-под запущенного тора.

In [66]:
import socks
import socket
socks.set_default_proxy(socks.SOCKS5, "localhost", 9150)
socket.socket = socks.socksocket

Посмотрим на свой новый ip-aдрес. 

In [67]:
checkIP()

Current IP Address: 178.32.59.160


При сопутствующем желании, [можно выяснить](http://ru.smart-ip.net/geoip) из какой страны в данный момент мы сидим в интернете.

Попробуем обратиться к мемохранилищу с нового ip-адреса.

In [69]:
data_row = getMemeData('http://knowyourmeme.com/memes/doge')

for key, value in data_row.items():
    print(key.capitalize()+":", str(value)[:200], end='\n\n')

Name: Doge

Status: Confirmed

Type: Animal

Origin_year: 2013

Origin_place: Tumblr

Date_added: 2018-10-23T01:43:59-04:00

Views: 13072407

Videos: 62

Photos: 1663

Comments: 917

Tags: animal, dog, shiba inu, shibe, such doge, super shibe, japanese, super, tumblr, much, very, many, comic sans, photoshop meme, such, shiba, shibe doge, doges, dogges, reddit, comic sans ms, tumblr meme

About: Doge is a slang term for "dog" that is primarily associated with pictures of Shiba Inus (nicknamed "Shibe") and internal monologue captions on Tumblr. These photos may be photoshopped to change the do

Origin: The use of the misspelled word "doge" to refer to a dog dates back to June 24th, 2005, when it was mentioned in an episode of Homestar Runner's puppet show. In the episode titled "Biz Cas Fri 1"[2], H

Other_text: Identity On February 23rd, 2010, Japanese kindergarten teacher Atsuko Sato posted several photos of her rescue-adopted Shiba Inu dog Kabosu to her personal blog.[38] Among the ph

Бан снят. Стражники мемов ничего не заподозрили и пустили нас в сокровищницу. Чашу нашего респонса снова переполняет контент. Через силу мы обрели мощь. 

При желании, можно выяснить одну занимательную вещь: при базовых настройках, Тор-браузер меняет ip каждые 10 минут. Но что делать, если сервер банит нас быстрее?  Всё просто, в папке, куда был установлен Tor найдём файлик с настройками под названием torrc (на маке он лежит по адресу `~/Library/Application Support/TorBrowser-Data/torrc`, если не получится найти - добро пожаловать [сюда](https://tor.stackexchange.com/questions/11866/cant-find-torrc-file-on-mac)) и отредактируем его. Добавим строки: 

```
CircuitBuildTimeout 10
LearnCircuitBuildTimeout 0
MaxCircuitDirtiness 10
```

Минимально возможный период для обновления ip составляет 10 секунд. Установим туда эту цифру и попробуем поиграться. 

In [70]:
for i in range(10):
    checkIP()
    time.sleep(5)

Current IP Address: 71.19.144.106
Current IP Address: 71.19.144.106
Current IP Address: 109.70.100.27
Current IP Address: 109.70.100.27
Current IP Address: 185.220.100.254
Current IP Address: 185.220.100.254
Current IP Address: 185.220.102.7
Current IP Address: 185.220.102.7
Current IP Address: 92.62.139.103
Current IP Address: 92.62.139.103


Действительно, смена ip происходит примерно раз в 10 секунд. Для наших целей по скачке мемов было достаточно и базовых настроек. Бан наступал примерно через 20 минут после начала работы кода. 

1. Открываем браузер;
2. Запускаем кусок кода с подгрузкой библиотек;
3. Запускаем цикл по мемам 
4. .....
5. Profit

In [55]:
final_df = pd.DataFrame(columns=['name', 'status', 'type', 'origin_year', 'origin_place',
                                 'date_added', 'views', 'videos', 'photos', 'comments', 
                                 'tags', 'about', 'origin', 'other_text'])


for page_number in tqdm_notebook(range(1075), desc='Pages'):
    # собрали хрефы с текущей страницы
    meme_links = getPageLinks(page_number)  
    for meme_link in tqdm_notebook(meme_links, desc='Memes', leave=False):
        # иногда с первого раза страничка не парсится
        for i in range(5):
            try:
                # пытаемся собрать по мему немного даты
                data_row = getMemeData(meme_link)           
                # и закидываем её в таблицу
                final_df = final_df.append(data_row, ignore_index=True)  
                # если всё получилось - выходим из внутреннего цикла
                break
            except:
                # Иначе, пробуем еще несколько раз, пока не закончатся попытки
                continue
                
final_df.to_csv('MEMES.csv')

Все мемы в наших руках. Можно приступать к варке фичей и моделированию. 

Теперь весь код для париснга можно перевести на рельсы этого небольшого скрипта, заменив все реквесты на торовские и менять IP адрес через каждые несколько реквестов к серверу, на который внезапно обрушится огромное число запросов из всех уголков света.

Напоследок, хотелось бы сказать пару слов о парсинге вообще и при помощи Тора в частности. Добывать себе данные - это стильно, модно и в принципе интересно, можно получить датасеты, которых еще никто никогда не обрабатывал, сделать что-то новое, посмотреть, наконец, на все мемы мира сразу. Однако не стоит забывать, что ограничения, введенные сервером, в том числе баны, появились не просто так, а в целях защиты сайта от недоброжелательных ковровых бомбардировок запросами и DDoS-атак. К чужому труду стоит относится с уважением, и даже если у сервера никакой защиты нет, - это еще не повод неограниченно забрасывать его реквестами, особенно если это может привести к его отключению - [уголовное наказание](http://sd-company.su/article/security/ddosataka-ugolovnaya-otvetstvennost) никто не отменял. Успешных и безопасных вам исследований!

Авторы: Ульянкин Филипп, Сергеев Дмитрий

## Почиташки 

* [Годная книга](https://github.com/FUlyankin/Parsers/blob/master/Ryan_Mitchell_Web_Scraping_with_Python-_Collecting_Data_from_the_Modern_Web_2015.pdf) про парсинг на английском языке. 
* [Неплохая инструкция](https://jarroba.com/anonymous-scraping-by-tor-network/) о самостоятельном парсинге через Tor без использования чужих готовых классов. 
* [Димин репозиторий](https://github.com/DmitrySerg/memology) с исследованием мемов.
* [Филин репозиторий](https://fulyankin.github.io/Parsers/) с ещё парой хорошо расписаных блокнотов с парсерами. Репозиторий изначально делался как страничка факультатива для студентов. 
* [Оригинальный кодекс](http://starwars.wikia.com/wiki/Code_of_the_Sith) адепта тёмной стороны силы.